# 概要

## 目的
今日（2025/03/19）から作成している。zip系で使えるutilsを全て保存

# 使用可能なutils

### 1. [calculate_zip_size]
- zipファイルのサイズを計算する




# 使用可能なutils

## calculate_zip_size

In [ ]:
import zipfile
from pathlib import Path
import requests
from tqdm import tqdm


def calculate_zip_size(zip_path):
    """
    指定したZIPファイルの展開前の総データサイズを計算する。

    Args:
        zip_path (str): ZIPファイルのパス

    Returns:
        tuple: (MB単位のサイズ, GB単位のサイズ)

    Examples:
        size_mb, size_gb = calculate_zip_size(zip_path)
        print(f"ZIPファイルのデータ量: {size_mb:.2f} MB ({size_gb:.2f} GB), filepath:{zip_path}")
    """
    total_size = 0

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for file_info in zip_ref.infolist():
            total_size += file_info.file_size

    size_mb = total_size / (1024 * 1024)
    size_gb = total_size / (1024 * 1024 * 1024)

    return size_mb, size_gb



def download_zip(url_zip_path: str, output_zip_path: Path):
    """
    指定されたURLからZIPファイルをダウンロードします。

    Args:
        url_zip_path (str): ダウンロードするZIPファイルのURL
        output_zip_path (Path): ダウンロードしたZIPファイルを保存するパス

    Examples:
        download_zip('http://example.com/file.zip', Path('./downloaded/file.zip'))
    """
    response = requests.get(url_zip_path, stream=True)
    response.raise_for_status()  # エラーチェック

    with output_zip_path.open("wb") as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)
    print(f"Downloaded: {output_zip_path}")


def unzip_all_files(zip_dir, extract_dir):
    """
    指定したディレクトリ内のすべての.zipファイルを解凍します。

    Parameters:
    zip_dir (str or Path): .zipファイルが格納されているディレクトリのパス
    extract_dir (str or Path): 解凍先のディレクトリのパス
    Examples:
        unzip_all_files('./zip_files/', './extracted_files/')
    """
    zip_dir = Path(zip_dir)
    extract_dir = Path(extract_dir)

    # zip_dir内のすべての.zipファイルを取得
    zip_files = zip_dir.glob('*.zip')

    # 各.zipファイルを解凍
    for zip_file in tqdm(zip_files, desc='Total Progress', unit='file'):
        with zipfile.ZipFile(zip_file, 'r') as z:
            # .zipファイル名と同じ名前のフォルダを作成して解凍
            extract_path = extract_dir / zip_file.stem
            extract_path.mkdir(parents=True, exist_ok=True)
            z.extractall(extract_path)
            print(f'{zip_file} を {extract_path} に解凍しました。')


# 実際の使用例
path = "/content/drive/MyDrive/AI_competition/vizwiz_vqa/data"
zip_paths = [str(p) for p in Path(path).glob("*.zip")]

for zip_path in zip_paths:
    size_mb, size_gb = calculate_zip_size(zip_path)
    print(f"ZIPファイルのデータ量: {size_mb:.2f} MB ({size_gb:.2f} GB), filepath:{zip_path}")


for original_zip_path in tqdm(zip_paths):
    zip_filename = Path(original_zip_path).name
    save_dir = Path("./data")
    output_zip_path = save_dir / zip_filename

    download_zip(original_zip_path, output_zip_path)


unzip_all_files(path, path)